# Create PostgreSQL Docker 

In [14]:
%time
!mkdir -p init
!docker run -d \
    -p 5433:5432 \
    --name optuna_pg \
    -e POSTGRES_USER=admin \
    -e POSTGRES_DB=optuna \
    -v $PWD/init:/docker-entrypoint-initdb.d \
    postgres

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
c4083923c29bfa7c59092169adfc302e99ef6fe15d67e17f804f558f0920d8b2


In [1]:
!docker ps | grep optuna_pg

c4083923c29b        postgres              "docker-entrypoint.s…"   8 minutes ago       Up 8 minutes        0.0.0.0:5433->5432/tcp                           optuna_pg


In [2]:
!pgcli -p 5433 -d optuna -h localhost -U admin  -l

List of databases
+-----------+-------+----------+------------+------------+-------------------+
| Name      | Owner | Encoding | Collate    | Ctype      | Access privileges |
+-----------+-------+----------+------------+------------+-------------------+
| optuna    | admin | UTF8     | en_US.utf8 | en_US.utf8 | <null>            |
| postgres  | admin | UTF8     | en_US.utf8 | en_US.utf8 | <null>            |
| template0 | admin | UTF8     | en_US.utf8 | en_US.utf8 | =c/admin          |
|           |       |          |            |            | admin=CTc/admin   |
| template1 | admin | UTF8     | en_US.utf8 | en_US.utf8 | =c/admin          |
|           |       |          |            |            | admin=CTc/admin   |
+-----------+-------+----------+------------+------------+-------------------+
SELECT 4


### test connection

In [12]:
from sqlalchemy import create_engine
pg_url = 'postgresql+pg8000://admin@localhost:5433/optuna'
engine = create_engine(pg_url)
engine.table_names()

[]

### distributed optuna

In [19]:
import optuna
from sklearn.ensemble import RandomForestClassifier

In [16]:
study = optuna.create_study(study_name='sklearn_rf_ieee_fraud', storage=pg_url)

[I 2020-01-20 16:15:53,064] A new study created with name: sklearn_rf_ieee_fraud


### data

In [22]:
import pandas as pd
import numpy as np

df = pd.read_csv('train_transaction.csv', low_memory=True)

In [23]:
for i in range(1,10):
    df['M' + str(i)] = df['M' + str(i)].eq('T').mul(1)

In [24]:
df = df.drop(columns=['P_emaildomain', 'R_emaildomain'], axis=1)

In [25]:
df = pd.concat([df, pd.get_dummies(df['ProductCD'], drop_first=True)], axis=1)
df = df.drop(columns=['ProductCD'], axis=1)

In [26]:
df = pd.concat([df, pd.get_dummies(df['card4'])], axis=1)
df = df.drop(columns=['card4'], axis=1)

In [27]:
df = pd.concat([df, pd.get_dummies(df['card6'])], axis=1)
df = df.drop(columns=['card6'], axis=1)

In [28]:
for c in df.columns:
    df[c] = df[c].fillna(df[c].mean())

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
# SPLIT DATA FOR CROSS VALIDATION
y = df['isFraud']
X = df.drop(columns = ['isFraud'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=12,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

### objective function

In [34]:
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn import metrics

def rf_ojbective(trial):
    
    # Random Forest Params
    n_estimators = trial.suggest_int('n_estimators', 25, 250)
    criterion = trial.suggest_categorical('criterion', ['gini','entropy'])
    max_depth = trial.suggest_int('max_depth',1,20)
    min_samples_split = trial.suggest_int('min_samples_split',2,10)
    
    # Induction
    clf = RandomForestClassifier(
        n_estimators = n_estimators, 
        criterion = criterion,
        max_depth = max_depth, 
        min_samples_split = min_samples_split,
        n_jobs = 12,
        random_state = 0)
    clf.fit(X_train, y_train)
    
    # Test
    fpr, tpr, thresholds = metrics.roc_curve(y_test, [y_hat[1] for y_hat in clf.predict_proba(X_test)], pos_label=1)
    result = 1 - metrics.auc(fpr, tpr)
    return result

### optimize

In [38]:
%%time
study.optimize(rf_ojbective, n_trials=100000)

[I 2020-01-20 16:48:48,209] Finished trial#8 resulted in value: 0.14263984225715864. Current best value is 0.07414497928728958 with parameters: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 244}.
[I 2020-01-20 16:49:07,925] Finished trial#9 resulted in value: 0.170449387598395. Current best value is 0.07414497928728958 with parameters: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 244}.
[I 2020-01-20 16:50:18,854] Finished trial#10 resulted in value: 0.12792193922306339. Current best value is 0.07414497928728958 with parameters: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 244}.
[I 2020-01-20 16:51:16,145] Finished trial#11 resulted in value: 0.136426148755735. Current best value is 0.07414497928728958 with parameters: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 244}.
[I 2020-01-20 16:51:38,966] Finished trial#12 resulted in value: 0.1700166

[I 2020-01-20 17:39:25,161] Finished trial#42 resulted in value: 0.1476737573484762. Current best value is 0.07412447601397609 with parameters: {'n_estimators': 250, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 2}.
[I 2020-01-20 17:39:33,037] Finished trial#43 resulted in value: 0.2741518425206537. Current best value is 0.07412447601397609 with parameters: {'n_estimators': 250, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 2}.
[I 2020-01-20 17:41:52,785] Finished trial#44 resulted in value: 0.07432653235092856. Current best value is 0.07412447601397609 with parameters: {'n_estimators': 250, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 2}.
[I 2020-01-20 17:44:07,530] Finished trial#45 resulted in value: 0.07474669988422677. Current best value is 0.07412447601397609 with parameters: {'n_estimators': 250, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 2}.
[I 2020-01-20 17:46:17,758] Finished trial#46 resulted in value: 0.078

[I 2020-01-20 18:45:55,483] Finished trial#76 resulted in value: 0.0755880487882401. Current best value is 0.07412447601397609 with parameters: {'n_estimators': 250, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 2}.


KeyboardInterrupt: 

In [73]:
import pandas as pd
import sqlalchemy as db
from sqlalchemy import create_engine

pg_url = 'postgresql+pg8000://admin@localhost:5433/optuna'
engine = create_engine(pg_url)
metadata = db.MetaData()
tables = engine.table_names()
tables

['version_info',
 'studies',
 'study_user_attributes',
 'study_system_attributes',
 'trials',
 'trial_user_attributes',
 'trial_system_attributes',
 'trial_params',
 'trial_values',
 'alembic_version']

In [74]:
def get_tbl(tbl):
    return pd.read_sql_query('select * from ' + tbl, engine)

In [77]:
for t in ['studies', 'trials','trial_params']:
    display(t, get_tbl(t))
    print('---------------------------\n')

'studies'

,study_id,study_name,direction
0,1,sklearn_rf_ieee_fraud,MINIMIZE


---------------------------



'trials'

,trial_id,study_id,state,value,datetime_start,datetime_complete
0,1,1,FAIL,NaN,2020-01-20 16:22:27.514063,2020-01-20 16:22:27.731792
1,2,1,FAIL,NaN,2020-01-20 16:23:08.662825,2020-01-20 16:23:08.883756
2,3,1,FAIL,NaN,2020-01-20 16:27:52.413196,2020-01-20 16:28:27.406747
3,4,1,COMPLETE,0.143193,2020-01-20 16:29:12.619799,2020-01-20 16:30:23.502020
4,5,1,COMPLETE,0.128107,2020-01-20 16:30:23.542472,2020-01-20 16:31:06.205273
...,...,...,...,...,...,...
73,74,1,COMPLETE,0.074638,2020-01-20 18:37:36.957851,2020-01-20 18:39:23.025868
74,75,1,COMPLETE,0.074402,2020-01-20 18:39:23.051571,2020-01-20 18:41:35.253074
75,76,1,COMPLETE,0.074379,2020-01-20 18:41:35.279743,2020-01-20 18:43:51.322061
76,77,1,COMPLETE,0.075588,2020-01-20 18:43:51.347771,2020-01-20 18:45:55.457952


---------------------------



'trial_params'

,param_id,trial_id,param_name,param_value,distribution_json
0,1,1,n_estimators,101.0,"{""name"": ""IntUniformDistribution"", ""attributes..."
1,2,1,criterion,1.0,"{""name"": ""CategoricalDistribution"", ""attribute..."
2,3,1,max_depth,2.0,"{""name"": ""IntUniformDistribution"", ""attributes..."
3,4,1,min_samples_split,6.0,"{""name"": ""IntUniformDistribution"", ""attributes..."
4,5,2,n_estimators,86.0,"{""name"": ""IntUniformDistribution"", ""attributes..."
...,...,...,...,...,...
307,308,77,min_samples_split,2.0,"{""name"": ""IntUniformDistribution"", ""attributes..."
308,309,78,n_estimators,236.0,"{""name"": ""IntUniformDistribution"", ""attributes..."
309,310,78,criterion,1.0,"{""name"": ""CategoricalDistribution"", ""attribute..."
310,311,78,max_depth,19.0,"{""name"": ""IntUniformDistribution"", ""attributes..."


---------------------------



In [44]:
import plaidml.keras
plaidml.keras.install_backend()